In [27]:
import csv
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mm
import sys
# sir_inference imports
from sir_model import FastProximityModel, patient_zeros_states
from ranking import csr_to_list
import os.path
from os import path
N=50000
## new try with 100 spreaders
N_patient_zero = 20;
lamb = 0.03;
mu = 0.05;
scale=1.0; # Easy Case
T=100;
seed=2
location="networks"
# SIR parameters
t1 = 10;
#seed=int();

In [28]:
## import sys
from time import time
import numpy as np
import pandas as pd
import os.path
from os import path
from scenario import Scenario_check_R
from sir_model import EpidemicModel, patient_zeros_states, symptomatic_individuals
from ranking import RANKINGS
log_fname="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb,seed)
csv_file=location+"/"+log_fname
print("seed = %d"%seed, flush=True)
np.random.seed(seed);
if path.exists(csv_file): print("Loading model from %s"%csv_file)
else:
    print("Could not find the file! Was looking for \n "+csv_file+"\n Bye-Bye")
    sys.exit()
print("Load Proximity model", flush=True)
tic = time()
model = EpidemicModel(initial_states=np.zeros(N), x_pos=np.zeros(N), y_pos=np.zeros(N))
model.load_transmissions(csv_file)
model.recover_probas = mu*np.ones(N)
print(f"Loading took {time()-tic:.1f}s",flush=True)
model.initial_states = patient_zeros_states(N, N_patient_zero)
model.time_evolution(model.recover_probas, model.transmissions, print_every=50)
t_max = len(model.transmissions)
print("Save plain dynamics", flush=True)
db = pd.DataFrame()
db["S"] = np.sum(model.states==0,axis=1)
db["I"] = np.sum(model.states==1,axis=1)
db["R"] = np.sum(model.states==2,axis=1)
db.to_csv("csv/Proximity_N%dK_T%d_s1_pz%d_mu%.2f_l%.2f_seed%d.csv"%(N/1000,T,N_patient_zero,mu,lamb,seed),
          index=False, sep="\t")
del db
model.initial_states = model.states[t1]
model.states = model.states[t1:]
model.transmissions = model.transmissions[t1:]

seed = 2
Loading model from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s2.csv
Load Proximity model
Loading transmissions from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s2.csv
Loading took 28.8s
Running SIR simulation
t = 0 / 100
t = 50 / 100
Save plain dynamics


In [29]:
# trac parameters
trac_tau = 5;
# MF parameters
MF_tau = 5;
MF_delta = 15;
# observation parameters
n_ranking = 100
p_untracked=0
#seed = int(sys.argv[2]);
#seeds for running [32,123,456]
#seed=int(sys.argv[1]);
################################################

intervention_options=dict(quarantine_time=T-t1)
observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)

In [31]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario_check_R
##### RANDOM SCENARIO #######
scenario_rnd = Scenario_check_R(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["random"]),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_rnd.run(t_max-t1,  print_every = 1)
print("Save random strategy", flush=True)
scenario_rnd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_rnd.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking),
          index=False, sep="\t")
del scenario_rnd

Using ranking_random to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49924, I:60, R:16 I+R:76
obser: S:100, I:0, R:0, I+R:0
free birds: 60
t = 1 / 90
state: S:49916, I:66, R:18 I+R:84
obser: S:200, I:0, R:0, I+R:0
free birds: 66
t = 2 / 90
state: S:49911, I:69, R:20 I+R:89
obser: S:300, I:0, R:0, I+R:0
free birds: 69
t = 3 / 90
state: S:49904, I:73, R:23 I+R:96
obser: S:400, I:0, R:0, I+R:0
free birds: 73
t = 4 / 90
state: S:49890, I:84, R:26 I+R:110
obser: S:500, I:0, R:0, I+R:0
free birds: 84
t = 5 / 90
state: S:49875, I:97, R:28 I+R:125
obser: S:600, I:0, R:0, I+R:0
free birds: 97
t = 6 / 90
state: S:49861, I:104, R:35 I+R:139
obser: S:700, I:3, R:1, I+R:4
free birds: 101
t = 7 / 90
state: S:49842, I:118, R:40 I+R:158
obser: S:800, I:5, R:1, I+R:6
free birds: 113
t = 8 / 90
state: S:49821, I:134, R:45 I+R:179
obser: S:899, I:9, R:1, I+R:10
free birds: 125
t = 9 / 90
state: S:49804, I:144, R:52 I+R:196
obser: S:999, I:13, R:4, I+R:17
free birds: 131
t

In [32]:
##### TRACING SCENARIO #######
scenario_trac = Scenario_check_R( model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["tracing"], tau=5),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_trac.run(t_max-t1, print_every = 1)
print("Save tracing strategy", flush=True)
scenario_trac.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac_t%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau),
                  index=False, sep="\t")
del scenario_trac

Using ranking_tracing to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49924, I:60, R:16 I+R:76
obser: S:100, I:0, R:0, I+R:0
free birds: 60
t = 1 / 90
state: S:49916, I:66, R:18 I+R:84
obser: S:200, I:0, R:0, I+R:0
free birds: 66
t = 2 / 90
state: S:49911, I:69, R:20 I+R:89
obser: S:300, I:0, R:0, I+R:0
free birds: 69
t = 3 / 90
state: S:49904, I:73, R:23 I+R:96
obser: S:400, I:0, R:0, I+R:0
free birds: 73
t = 4 / 90
state: S:49890, I:84, R:26 I+R:110
obser: S:500, I:0, R:0, I+R:0
free birds: 84
t = 5 / 90
state: S:49875, I:97, R:28 I+R:125
obser: S:600, I:0, R:0, I+R:0
free birds: 97
t = 6 / 90
state: S:49862, I:104, R:34 I+R:138
obser: S:700, I:4, R:0, I+R:4
free birds: 100
t = 7 / 90
state: S:49853, I:109, R:38 I+R:147
obser: S:788, I:12, R:6, I+R:18
free birds: 97
t = 8 / 90
state: S:49829, I:127, R:44 I+R:171
obser: S:876, I:19, R:14, I+R:33
free birds: 108
t = 9 / 90
state: S:49811, I:140, R:49 I+R:189
obser: S:958, I:34, R:24, I+R:58
free birds: 

In [22]:
##### MF SCENARIO #######
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario_check_R
scenario_MF = Scenario_check_R(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["backtrack"], 
                         algo="MF", init="all_S", tau=5, delta=10), 
    observation_options=observation_options,
    intervention_options=intervention_options,
)
scenario_MF.run(t_max-t1, print_every = 1)
print("Save MF strategy", flush=True)
scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,MF_tau,MF_delta),
                  index=False, sep="\t")
del scenario_MF
# 1h01min per round
print("End seed", flush=True)

Using ranking_backtrack to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49900, I:96, R:4 I+R:100
obser: S:100, I:0, R:0, I+R:0
free birds: 96
t = 1 / 90
state: S:49882, I:113, R:5 I+R:118
obser: S:198, I:2, R:0, I+R:2
free birds: 111
t = 2 / 90
state: S:49861, I:134, R:5 I+R:139
obser: S:298, I:2, R:0, I+R:2
free birds: 132
t = 3 / 90
state: S:49839, I:154, R:7 I+R:161
obser: S:398, I:2, R:0, I+R:2
free birds: 152
t = 4 / 90
state: S:49822, I:170, R:8 I+R:178
obser: S:498, I:2, R:0, I+R:2
free birds: 168
t = 5 / 90
state: S:49792, I:198, R:10 I+R:208
obser: S:598, I:2, R:0, I+R:2
free birds: 196
t = 6 / 90
state: S:49762, I:226, R:12 I+R:238
obser: S:698, I:10, R:1, I+R:11
free birds: 216
t = 7 / 90
state: S:49733, I:252, R:15 I+R:267
obser: S:797, I:21, R:1, I+R:22
free birds: 232
t = 8 / 90
state: S:49702, I:278, R:20 I+R:298
obser: S:896, I:33, R:1, I+R:34
free birds: 246
t = 9 / 90
state: S:49673, I:306, R:21 I+R:327
obser: S:995, I:42, R:1, I+R:43


In [26]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=50/N
psus = 0.55
pautoinf = 1e-10
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


Using step_scenario to rank
Using seed=2
Selecting 0/50000 untracked individuals
SCENARIO BP
winBP: (S,I,R): (49909.3, 86.4, 4.3), seeds: 90.7, ll: -29801.0
t = 0 / 90
state: S:49900, I:96, R:4 I+R:100
obser: S:98, I:2, R:0, I+R:2
free birds: 94
SCENARIO BP
1 obs 100
winBP: (S,I,R): (49898.0, 93.0, 9.0), seeds: 89.8, ll: -29816.7
t = 1 / 90
state: S:49884, I:112, R:4 I+R:116
obser: S:198, I:2, R:0, I+R:2
free birds: 110
SCENARIO BP
2 obs 100
winBP: (S,I,R): (49889.2, 97.0, 13.8), seeds: 86.7, ll: -29819.9
t = 2 / 90
state: S:49861, I:135, R:4 I+R:139
obser: S:298, I:2, R:0, I+R:2
free birds: 133
SCENARIO BP
3 obs 100
winBP: (S,I,R): (49881.0, 100.4, 18.6), seeds: 83.6, ll: -29833.3
t = 3 / 90
state: S:49842, I:154, R:4 I+R:158
obser: S:397, I:3, R:0, I+R:3
free birds: 151
SCENARIO BP
4 obs 100
winBP: (S,I,R): (49872.2, 104.2, 23.6), seeds: 80.9, ll: -29832.0
t = 4 / 90
state: S:49818, I:176, R:6 I+R:182
obser: S:480, I:20, R:0, I+R:20
free birds: 156
SCENARIO BP
5 obs 100
winBP: (S,I,R

KeyboardInterrupt: 